Запускал локально

In [ ]:
!pip install pandas sqlalchemy psycopg2

   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   --------------------------- ------------ 0.8/1.2 MB 2.8 MB/s eta 0:00:01
   ---------------------------------------- 1.2/1.2 MB 2.6 MB/s eta 0:00:00


    tinycss2 (>=1.1.0<1.2) ; extra == 'css'
             ~~~~~~~~^

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

In [ ]:
user = 'postgres'
password = 'postgres'
host = 'localhost'
port = '5432'
database = 'RoadWorksAnalyzer'

In [ ]:
connection_string = f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}'
engine = create_engine(connection_string)

In [ ]:
df = pd.read_csv('Moscow_all_processed_data.csv', sep='^')
df.head()

,global_id,Address,AdmArea,District,WorksType,WorksStatus,Customer,ActualBeginDate,WorksBeginDate,ActualEndDate,PlannedEndDate,Lanes_closed,geoData,WorkYear
0,2639121078,"Фрунзенская наб., д.8",Центральный административный округ,район Хамовники,"Ремонтные работы, Работы ДЖКХ",идут,Unknown,2024-02-19,2024-02-19,2025-05-20,2025-05-20,1,"[[37.595787445, 55.732870129], [37.59622944, 5...",2025
1,2639494326,ул. Большая Академическая (ул. Валаамская - ул...,Северный административный округ,Тимирязевский район,"Ремонтные работы, Работы коммерческих организаций",закончены,Unknown,2024-01-15,2024-01-15,2024-12-08,2024-12-08,1,"[[37.560100034, 55.84447127], [37.560007415, 5...",2024
2,2639494348,ул. Новослободская (в сторону области) (ул. Па...,Центральный административный округ,Тверской район,"Ремонтные работы, Работы коммерческих организаций",закончены,Unknown,2024-05-29,2024-05-29,2024-11-19,2024-11-19,1,"[[37.597433083, 55.783436047], [37.597221136, ...",2024
3,2639494385,"ул. Маршала Тухачевского, д.15 - д.14, к.1",Северо-Западный административный округ,район Хорошёво-Мнёвники,"Ремонтные работы, Работы ДЖКХ",закончены,Unknown,2024-02-07,2024-02-07,2024-06-30,2024-06-30,1,"[[37.486243397, 55.781979222], [37.48678118, 5...",2024
4,2639494387,"Зелёный пр-кт, д.76",Восточный административный округ,район Новогиреево,"Ремонтные работы, Работы ДЖКХ",закончены,Unknown,2023-10-19,2023-10-19,2024-11-30,2024-11-30,1,"[[37.829881509, 55.74968833], [37.830577344, 5...",2024


In [ ]:
df.columns = df.columns.str.lower()

Скрипт для создания таблицы

In [ ]:
CREATE TABLE road_works (
    global_id BIGSERIAL, -- не primary key, потому что не уникальный
    address TEXT,
    admarea TEXT,
    district TEXT,
    workstype TEXT,
    worksstatus TEXT,
    customer TEXT,
    actualbegindate DATE,
    worksbegindate DATE,
    actualenddate DATE,
    plannedenddate DATE,
    lanes_closed INTEGER,
    geodata TEXT,
    workyear INTEGER
);

In [ ]:
db_settings = {
    'dbname': 'RoadWorksAnalyzer',
    'user': 'postgres',
    'password': 'postgres',
    'host': 'localhost',
    'port': '5432'
}

create_table_query = """
CREATE TABLE IF NOT EXISTS road_works (
    global_id BIGSERIAL,
    address TEXT,
    admarea TEXT,
    district TEXT,
    workstype TEXT,
    worksstatus TEXT,
    customer TEXT,
    actualbegindate DATE,
    worksbegindate DATE,
    actualenddate DATE,
    plannedenddate DATE,
    lanes_closed INTEGER,
    geodata TEXT,
    workyear INTEGER
);
"""
cursor = None
conn = None
try:
    conn = psycopg2.connect(**db_settings)
    cursor = conn.cursor()

    cursor.execute(create_table_query)
    conn.commit()

    print("Таблица 'road_works' успешно создана!")

except Exception as e:
    print(f"Ошибка: {e}")

finally:
    if cursor:
        cursor.close()
    if conn:
        conn.close()

Таблица 'road_works' успешно создана!


global_id не уникальное поле, нужно что-то с этим делать

UPD: понял почему, делали explode

In [ ]:
df['global_id'].duplicated().sum()

1500

Загрузка в бд. Можно переделать в классы-модели и работать только с ними (так правильнее)

In [ ]:
try:
    df.to_sql('road_works', engine, if_exists='append', index=False)
    print("Данные успешно загружены!")
except Exception as e:
    print(f"Ошибка загрузки: {e}")

Данные успешно загружены!


Чтение данных с пагинацией

In [ ]:
page_size = 10
page_number = 1

offset = (page_number - 1) * page_size

query = f"""
    SELECT *
    FROM road_works
    LIMIT {page_size} OFFSET {offset};
"""

try:
    df = pd.read_sql(query, engine)
    print(df)
except Exception as e:
    print(f"Ошибка: {e}")

    global_id                                            address  \
0  2639121078                              Фрунзенская наб., д.8   
1  2639494326  ул. Большая Академическая (ул. Валаамская - ул...   
2  2639494348  ул. Новослободская (в сторону области) (ул. Па...   
3  2639494385         ул. Маршала Тухачевского, д.15 - д.14, к.1   
4  2639494387                                Зелёный пр-кт, д.76   
5  2639494411                       Савеловская линия, д.7 - д.9   
6  2639494416                        Чечерский пр-д, д.28 - д.26   
7  2639494433  Нижний Кисловский пер. (Никитский б-р, д.8А - ...   
8  2639494435          ул. Мясищева (Филевский б-р, д.42 - д.39)   
9  2639494446  дублер Ленинского пр-кт (Ленинский пр-кт, д.67...   

                                   admarea                 district  \
0       Центральный административный округ          район Хамовники   
1          Северный административный округ      Тимирязевский район   
2       Центральный административный о